In [14]:
import pandas as pd
import numpy as np
import pickle
import os, time
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [15]:
# Reading CSV files, and merging all of them into a single DataFrame
CISIDS2017_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CICIDS2017ML"
CTU13_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CTU13ML"

CICIDS2017_df_list = []
CTU13_df_list = []

# Reading CICIDS2017 CSV files into a single DataFrame
for f in os.listdir(CISIDS2017_folder):
    file_path = os.path.join(CISIDS2017_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CICIDS2017_df_list.append(pd.read_csv(file_path))

# Reading CTU13 CSV files into a single DataFrame
for f in os.listdir(CTU13_folder):
    file_path = os.path.join(CTU13_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CTU13_df_list.append(pd.read_csv(file_path))

CICIDS2017_df, CTU13_df = pd.concat(CICIDS2017_df_list, ignore_index=True), pd.concat(CTU13_df_list, ignore_index=True)

Reading:  Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Morning.pcap_ISCX_Relabeled.csv
Reading:  Wednesday-workingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX_Relabeled.csv
Reading:  Monday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_Relabeled.csv
Reading:  Tuesday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX_Relabeled.csv
Reading:  CTU13_Normal_Traffic_Relabeled.csv
Reading:  CTU13_Attack_Traffic_Relabeled.csv


In [16]:
# QUICK PREPROCESSING. 
# Some classifiers do not like "infinite" (inf) or "null" (NaN) values.
CICIDS2017_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CICIDS2017_df.columns[CICIDS2017_df.isna().any()]))
CICIDS2017_df.dropna(inplace=True)

CTU13_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CTU13_df.columns[CTU13_df.isna().any()]))
CTU13_df.dropna(inplace=True)

Columns with problematic values:  [' Flow Packets/s']
Columns with problematic values:  []


In [17]:
CICIDS2017_df[' Label'].unique()

array(['BENIGN', 'PortScan', 'Bot', 'DoS slowloris', 'DoS Slowhttptest',
       'DoS Hulk', 'DoS GoldenEye', 'Heartbleed',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'DDoS', 'FTP-Patator', 'SSH-Patator',
       'Infiltration'], dtype=object)

In [18]:
CTU13_df[' Label'].unique()

array(['BENIGN', 'Bot'], dtype=object)

In [19]:
# Create a new column that unifies all malicious classes into a single class for binary classification
CICIDS2017_df['GT'] = np.where(CICIDS2017_df[' Label']=='BENIGN', 'Benign', 'Malicious')
CTU13_df['GT'] = np.where(CTU13_df[' Label']=='BENIGN', 'Benign', 'Malicious')

In [20]:
features = pd.Index([
    ' Flow Duration',
    ' Total Fwd Packets',
    ' Total Backward Packets',
    ' Total Length of Bwd Packets',
    ' Fwd Packet Length Max',
    ' Fwd Packet Length Min',
    ' Fwd Packet Length Mean',
    ' Fwd Packet Length Std',
    ' Bwd Packet Length Min',
    ' Bwd Packet Length Mean',
    ' Bwd Packet Length Std',
    ' Flow Packets/s',
    ' Flow IAT Mean',
    ' Flow IAT Std',
    ' Flow IAT Max',
    ' Flow IAT Min',
    ' Fwd IAT Mean',
    ' Fwd IAT Std',
    ' Fwd IAT Max',
    ' Fwd IAT Min',
    ' Bwd IAT Mean',
    ' Bwd IAT Std',
    ' Bwd IAT Max',
    ' Bwd IAT Min',
    ' Bwd PSH Flags',
    ' Fwd Header Length',
    ' Bwd Header Length',
    ' Bwd Packets/s',
    ' Min Packet Length',
    ' Max Packet Length',
    ' Packet Length Mean',
    ' Packet Length Std',
    ' Packet Length Variance',
    ' SYN Flag Count',
    ' RST Flag Count',
    ' ACK Flag Count',
    ' Down/Up Ratio',
    ' Average Packet Size',
    ' Avg Fwd Segment Size',
    ' Avg Bwd Segment Size',
    ' Init_Win_bytes_backward',
    ' act_data_pkt_fwd',
    ' Active Std',
    ' Active Max',
    ' Active Min',
    ' Idle Std',
    ' Idle Max',
    ' Idle Min'
])

In [21]:
# Train a Binary SVM model on CTU13 dataset
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])

start = time.time()
svmClf_bin_ctu13 = LinearSVC()
svmClf_bin_ctu13.fit(train_ctu13[features], train_ctu13['GT'])
end = time.time() - start
print("Training time (CTU13 Binary): ", end)

# Save the binary SVM model trained on CTU13 dataset
with open('../Pickle Files/SVMBinaryCTU13.pkl', 'wb') as file:
    pickle.dump(svmClf_bin_ctu13, file)

/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Training time (CTU13 Binary):  12.577931880950928


/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [22]:
# Train a Binary SVM model on CICIDS2017 dataset
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

start = time.time()
svmClf_bin_cicids = LinearSVC()
svmClf_bin_cicids.fit(train_cicids[features], train_cicids['GT'])
end = time.time() - start
print("Training time (CICIDS2017 Binary): ", end)

# Save the binary SVM model trained on CICIDS2017 dataset
with open('../Pickle Files/SVMBinaryCICIDS2017.pkl', 'wb') as file:
    pickle.dump(svmClf_bin_cicids, file)

/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Training time (CICIDS2017 Binary):  808.4376373291016


/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [23]:
# Train a Multiclass SVM model on CICIDS2017 dataset
start = time.time()
svmClf_multi_cicids = LinearSVC()
svmClf_multi_cicids.fit(train_cicids[features], train_cicids[' Label'])
end = time.time() - start
print("Training time (CICIDS2017 Multiclass): ", end)

# Save the multiclass SVM model trained on CICIDS2017 dataset
with open('../Pickle Files/SVMMulticlassCICIDS2017.pkl', 'wb') as file:
    pickle.dump(svmClf_multi_cicids, file)

/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Training time (CICIDS2017 Multiclass):  8600.66914319992


/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [24]:
# Load the binary SVM model trained on CTU13 dataset
with open('../Pickle Files/SVMBinaryCTU13.pkl', 'rb') as file:
    svmClf_bin_ctu13 = pickle.load(file)
    
# Test on CTU13 dataset
predictions_bin_ctu13 = svmClf_bin_ctu13.predict(test_ctu13[features])
print("Acc (CTU13 Binary): {:3f}".format(accuracy_score(test_ctu13['GT'], predictions_bin_ctu13)))
print("F1-score (CTU13 Binary): {:3f}".format(f1_score(test_ctu13['GT'], predictions_bin_ctu13, pos_label='Malicious')))
pd.crosstab(test_ctu13['GT'], predictions_bin_ctu13, rownames=['True'], colnames=['Pred'])

Acc (CTU13 Binary): 0.691501
F1-score (CTU13 Binary): 0.506484


Pred,Benign,Malicious
True,,
Benign,19667,1659
Malicious,9720,5839


In [25]:
# Load the binary SVM model trained on CICIDS2017 dataset
with open('../Pickle Files/SVMBinaryCICIDS2017.pkl', 'rb') as file:
    svmClf_bin_cicids = pickle.load(file)

# Test on CICIDS2017 dataset
predictions_bin_cicids = svmClf_bin_cicids.predict(test_cicids[features])
print("Acc (CICIDS2017 Binary): {:3f}".format(accuracy_score(test_cicids['GT'], predictions_bin_cicids)))
print("F1-score (CICIDS2017 Binary): {:3f}".format(f1_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
pd.crosstab(test_cicids['GT'], predictions_bin_cicids, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Binary): 0.745861
F1-score (CICIDS2017 Binary): 0.578732


Pred,Benign,Malicious
True,,
Benign,646220,262308
Malicious,25162,197461


In [26]:
# Load the multiclass SVM model trained on CICIDS2017 dataset
with open('../Pickle Files/SVMMulticlassCICIDS2017.pkl', 'rb') as file:
    svmClf_multi_cicids = pickle.load(file)

# Test on CICIDS2017 dataset
predictions_multi_cicids = svmClf_multi_cicids.predict(test_cicids[features])
print("Acc (CICIDS2017 Multiclass): {:3f}".format(accuracy_score(test_cicids[' Label'], predictions_multi_cicids)))
print("F1-score (CICIDS2017 Multiclass): {:3f}".format(f1_score(test_cicids[' Label'], predictions_multi_cicids, average='macro')))
pd.crosstab(test_cicids[' Label'], predictions_multi_cicids, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Multiclass): 0.743493
F1-score (CICIDS2017 Multiclass): 0.208552


Pred,BENIGN,Bot,DDoS,DoS GoldenEye,DoS Hulk,DoS Slowhttptest,DoS slowloris,FTP-Patator,Heartbleed,Infiltration,PortScan,SSH-Patator,Web Attack � Brute Force,Web Attack � Sql Injection,Web Attack � XSS
True,,,,,,,,,,,,,,,
BENIGN,711468,110,7958,282,99746,1670,10352,237,7641,52,68352,85,90,57,428
Bot,362,0,0,0,197,0,0,0,0,0,222,0,0,0,0
DDoS,12954,0,38323,1,57,19,6,0,25,0,14,0,0,0,0
DoS GoldenEye,3351,10,2,443,228,16,0,0,0,138,0,0,2,5,0
DoS Hulk,52183,0,12558,0,26647,76,109,0,0,0,263,0,0,0,0
DoS Slowhttptest,1704,0,39,0,183,264,8,0,0,0,1,4,0,1,0
DoS slowloris,897,0,0,0,494,670,55,0,2,0,147,1,0,2,0
FTP-Patator,1624,0,0,0,580,0,0,958,0,0,9,0,0,0,0
Heartbleed,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
# Load the binary SVM model trained on CTU13 dataset
with open('../Pickle Files/SVMBinaryCTU13.pkl', 'rb') as file:
    svmClf_bin_ctu13 = pickle.load(file)

# Test on CICIDS2017 dataset
predictions_bin_cicids = svmClf_bin_ctu13.predict(test_cicids[features])
print("Acc (CTU13 to CICIDS2017): {:3f}".format(accuracy_score(test_cicids['GT'], predictions_bin_cicids)))
print("F1-score (CTU13 to CICIDS2017): {:3f}".format(f1_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
pd.crosstab(test_cicids['GT'], predictions_bin_cicids, rownames=['True'], colnames=['Pred'])

Acc (CTU13 to CICIDS2017): 0.558234
F1-score (CTU13 to CICIDS2017): 0.221468


Pred,Benign,Malicious
True,,
Benign,560372,348156
Malicious,151548,71075


In [30]:
# Load the binary SVM model trained on CICIDS2017
with open('../Pickle Files/SVMBinaryCICIDS2017.pkl', 'rb') as file:
    svmClf_bin = pickle.load(file)

# Test on CTU13 dataset
predictions_bin = svmClf_bin.predict(test_ctu13[features])
print("Acc (CICIDS2017 Binary to CTU13): {:3f}".format(accuracy_score(test_ctu13['GT'], predictions_bin)))
print("F1-score (CICIDS2017 Binary to CTU13): {:3f}".format(f1_score(test_ctu13['GT'], predictions_bin, pos_label='Malicious')))
pd.crosstab(test_ctu13['GT'], predictions_bin, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Binary to CTU13): 0.640450
F1-score (CICIDS2017 Binary to CTU13): 0.464897


Pred,Benign,Malicious
True,,
Benign,17862,3464
Malicious,9798,5761


In [31]:
# Load the multiclass SVM model trained on CICIDS2017
with open('../Pickle Files/SVMMulticlassCICIDS2017.pkl', 'rb') as file:
    svmClf_multi = pickle.load(file)

# Test on CTU13 dataset
predictions_multi = svmClf_multi.predict(test_ctu13[features])
print("Acc (CICIDS2017 Multi-class to CTU13): {:3f}".format(accuracy_score(test_ctu13[' Label'], predictions_multi)))
print("F1-score (CICIDS2017 Multi-class to CTU13): {:3f}".format(f1_score(test_ctu13[' Label'], predictions_multi, average='macro')))
pd.crosstab(test_ctu13[' Label'], predictions_multi, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Multi-class to CTU13): 0.551173
F1-score (CICIDS2017 Multi-class to CTU13): 0.051655


Pred,BENIGN,Bot,DDoS,DoS GoldenEye,DoS Hulk,DoS Slowhttptest,DoS slowloris,FTP-Patator,Heartbleed,Infiltration,PortScan,SSH-Patator,Web Attack � Brute Force,Web Attack � Sql Injection,Web Attack � XSS
True,,,,,,,,,,,,,,,
BENIGN,20307,19,0,172,532,14,40,28,104,14,93,2,1,0,0
Bot,10984,23,21,3,812,159,26,37,171,6,3243,7,42,24,1
